<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

# <center>Árboles de decisión</center>

En este ejercicio de laboratorio, aprenderá un algoritmo de aprendizaje automático muy utilizado: el árbol de decisión. Empleará este algoritmo de clasificación para armar un modelo de datos históricos de pacientes y su reacción a diferentes medicamentos. Luego utilizará al árbol de decisión entrenado para predecir la clase de un paciente desconocido o hallar un fármaco apropiado para un nuevo paciente.

Importe las siguientes bibliotecas:
<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
    <li> <b>DecisionTreeClassifier</b> from <b>sklearn.tree</b> </li>
</ul>

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

### Acerca del conjunto de datos
Suponga que es un investigador médico que reúne datos para un estudio. Ha recogido datos de un conjunto de pacientes, todos los cuales padecen la misma enfermedad. Durante su tratamiento, cada paciente respondió a 5 medicamentos: Fármaco A [Drug A], Fármaco B [Drug B], Fármaco c [Drug c], Fármaco x [Drug x] y Fármaco y [Drug y].
Parte de su trabajo es armar un modelo para averiguar qué fármaco podría ser apropiado para un futuro paciente que padezca la misma enfermedad. Los conjuntos de características de este conjunto de datos son Edad [Age], Sexo [Sex], Presión arterial [Blood Pressure], y Colesterol [Cholesterol] de los pacientes, y el objetivo es el fármaco al que respondió cada paciente.
Es una muestra de clasificador binario, y puede utilizar la parte de entrenamiento del conjunto de datos para construir un árbol de decisión, y luego usarlo para predecir la clase de un paciente desconocido o para recetárselo a un nuevo paciente.


### Descarga de datos
Usaremos !wget para descargar los datos de IBM Object Storage.

In [ ]:
!wget -O drug200.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/drug200.csv

__¿Sabía usted?__ Al usar aprendizaje automático, es probable que trabaje con grandes conjuntos de datos. Como empresa, ¿dónde puede alojar sus datos? IBM ofrece una oportunidad inigualable para empresas, con 10 TB de almacenamiento en IBM Cloud Object Storage: [Regístrese ahora gratis](http://cocl.us/ML0101EN-IBM-Offer-CC)

Ahora usemos la hoja de datos de pandas para leer los datos:

In [ ]:
my_data = pd.read_csv("drug200.csv", delimiter=",")
my_data[0:5]

## Práctica 
¿Cuál es el tamaño de los datos?

In [ ]:
# write your code here




## Preprocesamiento

Usando <b>my_data</b> como los datos de Drug.csv que lee pandas, declare las siguientes variables: 
<br>
<ul>
    <li> <b> X </b> como <b> matriz de características [Feature Matrix] </b> (datos de my_data) </li>

    
    <li> <b> y </b> como <b> vector de respuesta (objetivo) </b> </li>


   
</ul>

Elimine la columna que contiene el nombre objetivo, ya que no contiene valores numéricos.

In [ ]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

Como se habrá dado cuenta, algunas características de este conjunto de datos son categóricas, tales como __Sexo [Sex]__ y __Presión arterial [BP]__. Lamentablemente, los árboles de decisión de Sklearn no manejan variables categóricas. No obstante, podemos convertir esas características en valores numéricos. __pandas.get_dummies()__ convierte variables categóricas en variables simuladas o indicadoras.

In [ ]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]


Ahora podemos llenar la variable objetivo.

In [ ]:
y = my_data["Drug"]
y[0:5]

---
## Preparación del árbol de decisión
Usaremos <b>división de entrenamiento/prueba (train/test split)</b> en nuestro <b>árbol de decisión</b>. Importemos la función <b>train_test_split</b> de <b>sklearn.cross_validation</b>.

In [ ]:
from sklearn.model_selection import train_test_split

Ahora <b> train_test_split </b> arrojará 4 parámetros. Los llamaremos:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>
La función <b> train_test_split </b> necesitará los siguientes parámetros: <br>
X, y, test_size=0.3, and random_state=3. <br> <br>
<b>X</b> e <b>y</b> son los arreglos que se necesitan antes de la división, <b>test_size</b> representa la razón matemática del conjunto de datos de prueba, y <b>random_state</b> garantiza que obtengamos las mismas divisiones.

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

## Práctica
Imprima la forma de X_trainset e y_trainset. Asegúrese de que las dimensiones coincidan.

In [ ]:
# your code



Imprima la forma de X_testset e y_testset. Asegúrese de que las dimensiones coincidan.

In [ ]:
# your code



## Modelado
Primero crearemos una instancia del <b>DecisionTreeClassifier (clasificador de árbol de decisión)</b> llamado <b>drugTree (árbol de fármacos)</b>.<br>
Dentro del clasificador, especificamos entropía como criterio <i> criterion="entropy" </i> para que podamos ver la ganancia de información en cada nodo.

In [ ]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # it shows the default parameters

A continuación, ajustaremos los datos con la matriz de características de entrenamiento <b> X_trainset </b> y el vector de respuesta de entrenamiento <b> y_trainset </b>

In [ ]:
drugTree.fit(X_trainset,y_trainset)

## Predicción
Hagamos algunas <b>predicciones [predictions]</b> en el conjunto de datos de prueba y almacenémoslas en una variable llamada <b>predTree</b>.

In [ ]:
predTree = drugTree.predict(X_testset)

Puede imprimir en pantalla <b>predTree</b> e <b>y_testset</b> si desea realizar una comparación visual de la predicción con los valores reales.

In [ ]:
print (predTree [0:5])
print (y_testset [0:5])


## Evaluación
A continuación, importemos las __métricas [metrics]__ de sklearn y comprobemos la exactitud de nuestro modelo.

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

La función __puntuación de clasificación de exactitud [Accuracy classification score]__ calcula la exactitud de los subconjuntos: el conjunto de etiquetas predichas para una muestra debe coincidir exactamente con el conjunto de etiquetas correspondiente de y_true.
En la clasificación de etiquetas múltiples, la función arroja la exactitud del subconjunto. Si todo el conjunto de etiquetas predichas de una muestra coincide con el conjunto real de etiquetas, la precisión del subconjunto es de 1.0; si no, es de 0.0.


## Práctica 
¿Puede calcular la puntuación de exactitud sin sklearn?

In [ ]:
# your code here



## Representación visual
Representemos el árbol en forma visual.

In [ ]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = my_data.columns[0:5]
targetNames = my_data["Drug"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

## ¿Desea saber más?

IBM SPSS Modeler es una plataforma de análisis completa que tiene muchos algoritmos de aprendizaje automático. Ha sido diseñada para aportar inteligencia predictiva a las decisiones que toman personas, grupos, sistemas, su empresa como conjunto. Este curso le permite acceder a una evaluación gratuita, disponible en este enlace: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

También puede usar Watson Studio para ejecutar estos cuadernos más rápido con conjuntos de datos más grandes. Watson Studio es la solución de IBM en la nube número uno para científicos de datos, construida por científicos de datos. Con los cuadernos Jupyter, RStudio, Apache Spark y otras bibliotecas populares preempaquetadas en la nube, Watson Studio hace posible que los científicos de datos colaboren en sus proyectos sin necesidad de instalar nada. Súmese hoy mismo a la comunidad de usuarios de Watson Studio, que crece cada día más, con una cuenta gratuita en [Watson Studio](https://cocl.us/ML0101EN_DSX)

### ¡Gracias por completar esta lección!

Cuaderno creado por: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). Este cuaderno y su código fuente se difunden de conformidad con los términos de la [Licencia de MIT](https://bigdatauniversity.com/mit-license/).​